## Generate data from Players
Generate and save games from each player vs random or telepath opponents to csv format files

In [1]:
from Lib.RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from Lib.Telepathic_player import telepath
from Lib.RPS_encoding import RPSpair_encode, RPSpair_decode
import os, shutil
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
def GenerateGames(player, opponent, game_count = 10, game_length = 1000, cheater = False):
    if(cheater):
        player = telepath
    
    filename = "Games/" + player.__name__ + "_vs_" + opponent.__name__ + "_game_"
    for i in range(0 , game_count):
        f = open(filename + str(i) + ".txt", "w")
        p1_prev_play = ""
        p2_prev_play = ""
        for _ in range(game_length):
            
            p2_play = opponent(p1_prev_play)
            if(cheater):
                p1_play = player(p2_play)
            else:
                p1_play = player(p2_prev_play)
            player_win = 0
            if (p1_play == "P" and p2_play == "R") or \
                (p1_play == "R" and p2_play == "S") or \
                (p1_play == "S" and p2_play == "P"):
                player_win = 1
            
            winning_play = 'R' if p2_play == 'S' else 'S' if p2_play == 'P' else 'P'
                
            f.write(p1_play + "," + p2_play + "," + str(player_win) + "," + winning_play + '\n')
            
            p1_prev_play = p1_play
            p2_prev_play = p2_play
        f.close()

## Import trained model to make guesses

In [6]:
def Get_Prediction(model, opponent_history, player_history):
    input_raw = [x + y if x != '' and y != '' else 'PR' for (x,y) in zip(player_history, opponent_history)]
    #print(input_raw)
    #format history for input
    input_eval = [RPSpair_encode(c) for c in input_raw]
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    try:
        return RPSpair_decode(predicted_id)[0]
    except:
        print(predicted_id)
        print("Guessing: " + RPSpair_decode(predicted_id))
        return "R"
    

def player(prev_play, opponent_history = [], play_history=['']):
    if len(play_history) > 1000: 
        opponent_history.clear()
        play_history.clear()
        play_history.append('')
    opponent_history.append(prev_play)
    #print("Actual: " + prev_play) 
    guess = Get_Prediction(model, opponent_history, play_history)
    #print(guess)
    play_history.append(guess)
    return guess

model = keras.models.load_model('Models/LSTM_ImbalancedTrainingAbbey_GameHistoryBased_Abbey') 
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 64)             576       
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 512)            1181696   
_________________________________________________________________
dense_3 (Dense)              (1, None, 10)             5130      
Total params: 1,187,402
Trainable params: 1,187,402
Non-trainable params: 0
_________________________________________________________________


## Clear Training Data Folder

In [3]:
#Run to clear training folder
folder = 'Games/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Generate Games
## Balanced Dataset (Effective for Quincy + Mrugesh)
Train to predict the response to quincy and mrugesh's next play based on the quincy and mrugesh's past plays

In [4]:
GenerateGames(random_player, quincy, game_count = 25)
GenerateGames(random_player, abbey, game_count = 25)
GenerateGames(random_player, kris, game_count = 25)
GenerateGames(random_player, mrugesh, game_count = 25)
GenerateGames(telepath, quincy, cheater = True, game_count = 10)
GenerateGames(telepath, abbey, cheater = True, game_count = 10)
GenerateGames(telepath, kris, cheater = True, game_count = 10)
GenerateGames(telepath, mrugesh, cheater = True, game_count = 10)

## Kris Dataset (Effective for Kris)
Train to predict the response to Kris' next play based on the LTSM's past plays

In [5]:
GenerateGames(random_player, abbey, game_count = 15)
GenerateGames(random_player, kris, game_count = 15)
GenerateGames(telepath, abbey, cheater = True, game_count = 5)
GenerateGames(telepath, kris, cheater = True, game_count = 5)

## Abbey Dataset (Effective for Abbey)
Train to predict the response to Abbey's next play based on the LTSM's past plays

In [4]:
GenerateGames(random_player, abbey, game_count = 40)
GenerateGames(telepath, abbey, cheater = True, game_count = 10)

In [8]:
GenerateGames(random_player, abbey, game_count = 15)
GenerateGames(telepath, abbey, cheater = True, game_count = 35)

In [8]:
#GenerateGames(random_player, abbey, game_count = 10)
GenerateGames(player, abbey, game_count = 20)
GenerateGames(telepath, abbey, cheater = True, game_count = 30)